Will wrap the tools of the chatbot of the 1.chatbot.ipynb, to build an MCP server that exposes 2 tools. You will use here the stdio transport and run the server in the provided local environment

## How can you create an MCP Server? - *Additional Note*

Let's take the example of a server that exposes tools. This server needs to handle two main requests from the client:
- listing all the tools
  
   <img src="images/server_list_tools.png" width="400">

- executing a particular tool
  
  <img src="images/server_call_tool.png" width="400">

There are two ways for creating an MCP server:
- **low-level implementation**: in this approach, you directly define and handle the various types of requests (`ListToolsRequest` and `CallToolRequest`). This approach allows you to customize every aspect of your server.
- **high-level implementation using `FastMCP`**: `FastMCP` is a high-level interface that makes building MCP servers faster and simpler. In this approach, you just focus on defining the tools as functions, and`FastMCP` handles all the protocol details.
  
You will use in this lesson `FastMCP`. If you'd like to learn more about the low-level approach, you can check out the resources at the end of this notebook.

## Building your MCP Server using `FastMCP`

You will build the files needed for your MCP server in the folder `2.1.mcp_project`. You will incrementally add or update more files to this project folder. The `2.1.mcp_project` folder is provided to you. You will first create the python file of the server `research_server.py` and save it in  `2.1.mcp_project` folder, then you'll prepare the environment to run the server.

To create your MCP server using `FastMCP`, you will initialize a `FastMCP` server labeled as `mcp` and decorating the functions with `@mcp.tool()`. `FastMCP` automatically generates the necessary MCP schema based on type hints and docstrings.


**Note**: The magic function `%%writefile 2.1.mcp_project/research_server.py` will not execute the code but it will save the content of the cell to the server file `research_server.py` in the directory: `2.1.mcp_project`. If you remove the magic function and run the cell, the code won't run in Jupyter notebook. You will run the server from the terminal in the next section. 

In [ ]:
%%writefile 2.1.mcp_project/research_server.py

import arxiv
import json
import os
from typing import List
from mcp.server.fastmcp import FastMCP


PAPER_DIR = "papers"

# Initialize FastMCP server
mcp = FastMCP("research")

@mcp.tool()
def search_papers(topic: str, max_results: int = 5) -> List[str]:
    """
    Search for papers on arXiv based on a topic and store their information.
    
    Args:
        topic: The topic to search for
        max_results: Maximum number of results to retrieve (default: 5)
        
    Returns:
        List of paper IDs found in the search
    """
    
    # Use arxiv to find the papers 
    client = arxiv.Client()

    # Search for the most relevant articles matching the queried topic
    search = arxiv.Search(
        query = topic,
        max_results = max_results,
        sort_by = arxiv.SortCriterion.Relevance
    )

    papers = client.results(search)
    
    # Create directory for this topic
    path = os.path.join(PAPER_DIR, topic.lower().replace(" ", "_"))
    os.makedirs(path, exist_ok=True)
    
    file_path = os.path.join(path, "papers_info.json")

    # Try to load existing papers info
    try:
        with open(file_path, "r") as json_file:
            papers_info = json.load(json_file)
    except (FileNotFoundError, json.JSONDecodeError):
        papers_info = {}

    # Process each paper and add to papers_info  
    paper_ids = []
    for paper in papers:
        paper_ids.append(paper.get_short_id())
        paper_info = {
            'title': paper.title,
            'authors': [author.name for author in paper.authors],
            'summary': paper.summary,
            'pdf_url': paper.pdf_url,
            'published': str(paper.published.date())
        }
        papers_info[paper.get_short_id()] = paper_info
    
    # Save updated papers_info to json file
    with open(file_path, "w") as json_file:
        json.dump(papers_info, json_file, indent=2)
    
    print(f"Results are saved in: {file_path}")
    
    return paper_ids

@mcp.tool()
def extract_info(paper_id: str) -> str:
    """
    Search for information about a specific paper across all topic directories.
    
    Args:
        paper_id: The ID of the paper to look for
        
    Returns:
        JSON string with paper information if found, error message if not found
    """
 
    for item in os.listdir(PAPER_DIR):
        item_path = os.path.join(PAPER_DIR, item)
        if os.path.isdir(item_path):
            file_path = os.path.join(item_path, "papers_info.json")
            if os.path.isfile(file_path):
                try:
                    with open(file_path, "r") as json_file:
                        papers_info = json.load(json_file)
                        if paper_id in papers_info:
                            return json.dumps(papers_info[paper_id], indent=2)
                except (FileNotFoundError, json.JSONDecodeError) as e:
                    print(f"Error reading {file_path}: {str(e)}")
                    continue
    
    return f"There's no saved information related to paper {paper_id}."



if __name__ == "__main__":
    # Initialize and run the server
    mcp.run(transport='stdio')

Overwriting mcp_project/research_server.py


## Setting up your Environment & Testing your Server

You'll now set up the environment that you will use to run and test the server. For that, you will use the `uv` tool, which helps you manage your Python environment: it automatically sets up the project files and manages the package dependencies.

**Terminal Instructions**

- To open the terminal, run the cell below.
- Navigate to the project directory and initiate it with `uv`:
    - `cd mcp_project`
    - `uv init`
-  Create virtual environment and activate it:
    - `uv venv`
    - `source .venv/bin/activate`
- Install dependencies:
    - `uv add mcp arxiv`
- Launch the inspector:
    - `npx @modelcontextprotocol/inspector uv run research_server.py`
    - If you get a message asking "need to install the following packages", type: `y`
- You will get a message saying that the inspector is up and running at a specific address. To open the inspector, click on that given address. The inspector will open in another tab.
- In the inspector UI, make sure to follow the video. You would need to specify under configuration the `Inspector Proxy Address`. Please check the "Inspector UI Instructions" below and run the last cell (after the terminal) to get the `Inspector Proxy Address` for your configurations.

In [ ]:
# start a new terminal // this will open a new terminal in the notebook environment so it's not needed to run on local
import os
from IPython.display import IFrame

IFrame(f"{os.environ.get('DLAI_LOCAL_URL').format(port=8888)}terminals/1", 
       width=600, height=768)

AttributeError: 'NoneType' object has no attribute 'format'